<a href="https://colab.research.google.com/github/Miss-Alpha/Miss-Alpha/blob/main/Copy_of_Oracle_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creating a RAG (Retrieval-Augmented Generation) with Oracle Generative AI Service**

The following codes were an attempt to reproduce [this document](https://github.com/oracle-devrel/technology-engineering/tree/main/ai-and-app-modernisation/ai-services/generative-ai-service/rag-genai/files).

There were two problems with the current document,
- **Not Thourugh:** they are so many libraries that haven't been imported in the code exmaples and we have to add them.
- **Outdated:** since the code was written, so many modules have changed.

Below I added the necessary libraries, and made the changes in the module names. I still kept the original code in the comments.

The only missing part is the `compartment_id` which is basically an Oracle ID. I believe being able to access that and by making few changes we are able to make the final model work.

I added the OpenAI part for illustrative purposes.

🌟 This notebook has been last updated on **5th of March.**

## 0 Requirements

In [1]:
!pip install llama-index
!pip install llama-index-vector-stores-qdrant
!pip install qdrant-client
!pip install sentence-transformers
!pip install llama-hub
!pip install llama-index-readers-web

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.8 MB/s eta 0:00:00


In [3]:
from langchain_community.llms import OCIGenAI

#from llama_index import VectorStoreIndex
from llama_index.core import VectorStoreIndex

#from llama_index import ServiceContext
from llama_index.core import ServiceContext

from llama_index.vector_stores.qdrant import QdrantVectorStore

#from llama_index.storage.storage_context import StorageContext
from llama_index.core import StorageContext

from qdrant_client import qdrant_client
from langchain.embeddings import SentenceTransformerEmbeddings

#from llama_hub.web.sitemap import SitemapReader
from llama_index.readers.web import SimpleWebPageReader

from llama_index.core import SummaryIndex

import openai

In [ ]:
web_address = 'https://mco.mycomplianceoffice.com/solutions/compliance-oversight?_gl=1*15vxh5d*_up*MQ..&gclid=Cj0KCQiA84CvBhCaARIsAMkAvkKpo1rs5ZpjH0F02I9wwiAenOtU89_o5DTLd3oxhoo1CIq8KdRXzF4aAg27EALw_wcB'
documents = SimpleWebPageReader(html_to_text=True).load_data(
    [web_address]
)

## 1 OCI GenAI Library

In [ ]:
# preview is not recognized
!pip install oci==2.118.1 #+preview.1.1697

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 13.3 MB/s eta 0:00:00
  Created wheel for python-pkcs11: filename=python_pkcs11-0.7.0-cp310-cp310-linux_x86_64.whl size=1309796 sha256=bf2b54a3d707b571d5efa1bdccaa635894f5a7d8d2ee634d07ea583d9ccd3ff6
  Stored in directory: /root/.cache/pip/wheels/4d/e8/c8/c7cae4f78cf7a1dee289c476922f407b026aa76ffddcd474a5
  Created wheel for circuitbreaker: filename=circuitbreaker-1.4.0-py3-none-any.whl size=7519 sha256=2be8124b8fb02179ca0769c00da27329f4a463c6764e0d6afd56d4add28b5868
  Stored in directory: /root/.cache/pip/wheels/3c/be/64/266b6ce

In [ ]:
client = qdrant_client.QdrantClient(location=":memory:")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
llm = OCIGenAI(model_id="cohere.command",service_endpoint="https://generativeai.aiservice.us-chicago-1.oci.oraclecloud.com",compartment_id = "ocid1.tenancy.oc1..aaaaaaaa5hwtrus75rauufcfvtnjnz3mc4xm2bzibbigva2bw4ne7ezkvzha",temperature=0.0)
system_prompt="As a support engineer, your role is to leverage the information in the context provided. Your task is to respond to queries based strictly on the information available in the provided context. Do not create new information under any circumstances. Refrain from repeating yourself. Extract your response solely from the context mentioned above. If the context does not contain relevant information for the question, respond with 'How can I assist you with questions related to the document?"
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1000, chunk_overlap=100, embed_model=embeddings,system_prompt=system_prompt)
vector_store = QdrantVectorStore(client=client, collection_name="ansh")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query("can i use OCI document understanding for files in french ?")
print(response)

ValidationError: 2 validation errors for OCIGenAI
temperature
  extra fields not permitted (type=value_error.extra)
__root__
  Could not authenticate with OCI client. Please check if ~/.oci/config exists. If INSTANCE_PRINCIPLE or RESOURCE_PRINCIPLE is used, Please check the specified auth_profile and auth_type are valid. (type=value_error)

More information can be found on [Langchain OCI document](https://python.langchain.com/docs/integrations/llms/oci_generative_ai).

## 2 OpenAI Approach

In [4]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

In [ ]:
index = SummaryIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What are Compliance Oversight solutions?")

In [ ]:
print(response)

Compliance Oversight solutions are essential tools for firms to manage their regulatory obligations effectively. These solutions facilitate communication of obligations, assessment of compliance status, corrective actions, and demonstration of compliance both internally and externally. They offer a structured framework for overseeing regulatory obligations, tracking changes, recording compliance, and identifying areas of concern. By visualizing obligations, integrating risk information, and providing real-time feedback, Compliance Oversight solutions streamline compliance management.


## 3 Building a Web Application

### 3.1 Quickstart

In [5]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.7 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=bdbbf0e45f2dd1705ad6b30888e03ebbaab82b80461afa95a9decbba1959331c
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0


In [ ]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch()

In [ ]:
import gradio as gr

def greet(question):
    return query_engine.query(question)

demo = gr.Interface(
    fn=greet,
    inputs=["text"],
    outputs=["text"],
    thumbnail = 'https://mco.mycomplianceoffice.com/hubfs/_mco2018/Logos/mco-color-logo.svg',

)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://058e1d02e9eeb59359.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 3.2 Gradio Block Function

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://48f49a6a168ef776cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 3.3 Gradio ChatInterface Function

#### 3.3.1 ChatInterface Without an LLM

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def greet(message, history):
    #return query_engine.query(message)
    return print(query_engine.query(message))

demo = gr.ChatInterface(fn=greet, examples=["What are Compliance Oversight solutions?", "hola", "merhaba"], title="MCO Assitant")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b48c4fdf354917fa0f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


#### 3.3.2 ChatInterface With an LLM

##### 3.3.2.1 Simple LLM Example

**Resources: **
- https://www.gradio.app/guides/creating-a-chatbot-fast

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    gpt_response = llm(history_langchain_format)
    return gpt_response.content

    #query_engine = index.as_query_engine()
    #response = query_engine.query(history_langchain_format)
    #return response.response


gr.ChatInterface(predict).launch()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://92883f26b8d71a62c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


###### GPT4

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

llm = ChatOpenAI(temperature=0.5, model='gpt-4-0125-preview')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    gpt_response = llm(history_langchain_format)
    return gpt_response.content

    #query_engine = index.as_query_engine()
    #response = query_engine.query(history_langchain_format)
    #return response.response


gr.ChatInterface(predict).launch()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ae63ae288dfc66399a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##### 3.3.2.2 RAG Example

**Resources: **
- https://www.gradio.app/guides/creating-a-chatbot-fast
- https://python.langchain.com/docs/expression_language/cookbook/retrieval

In [6]:
!pip install gradio

In [6]:
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel

In [7]:
!pip install langchain_openai

In [8]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00


In [9]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

In [10]:
MCO_info = "MCO provides integrated and comprehensive compliance software for financial services firms worldwide. Our mission is to help these firms effectively manage and mitigate compliance risk on a singular platform. MCO offers a unique combination of employee surveillance, transaction monitoring, and third-party oversight, ensuring that compliance obligations are not only met but exceeded.With clients in over 105 countries and employees around the world, we are committed to delivering affordable, easy-to-use, compliance technology, which we create and deliver with a passion for excellence."

In [11]:
vectorstore = FAISS.from_texts(
    #["harrison worked at kensho"], embedding=OpenAIEmbeddings()
    [MCO_info], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

#template = """Answer the question based only on the following context:
#{context}

#Question: {question}
#"""
#prompt = ChatPromptTemplate.from_template(template)

#model = ChatOpenAI()

In [12]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [13]:
#template = """Answer the question based only on the following context:
#{context}

#Question: {question}
#"""
template = """You're a helpful expert who has extensive knowledge on compliances. Answer the question based on the provided context but always respond to the question even if the context doesn't provide the information.
 :
{context}

Question: {question}
"""

ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [14]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [15]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [16]:
conversational_qa_chain.invoke(
    {
        #"question": "where did harrison work?",
        "question": "what is MCO?",
        "chat_history": [],
    }
)

AIMessage(content='MCO is a company that provides integrated and comprehensive compliance software for financial services firms worldwide. Their mission is to help these firms effectively manage and mitigate compliance risk on a singular platform.')

In [17]:
#from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

#llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    gpt_response = conversational_qa_chain.invoke(
        {
          "question": message,
          "chat_history": history_langchain_format
        })
    return gpt_response.content


gr.ChatInterface(predict,
                 title='MCO Assistant',
                 description='A generative AI assistant here to help you with your compliance related questions.',
                 #textbox=gr.Textbox(placeholder="Type your question here...", min_width=220),
                 examples=["What does MCO provide?", "C'est facile?", "Are tomatoes vegetables?"],
                 theme='Soft'
                 ).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7dc553b6339aa66da5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##### Liking/Disliking Feature

## ∞. Sandbox

In [19]:
import gradio as gr
import time

def echo(message, history, system_prompt, tokens):
    response = f"System prompt: {system_prompt}\n Message: {message}."
    for i in range(min(len(response), int(tokens))):
        time.sleep(0.05)
        yield response[: i+1]

demo = gr.ChatInterface(echo,
                        additional_inputs=[
                            gr.Textbox("You are helpful AI.", label="System Prompt"),
                            gr.Slider(10, 100)
                        ]
                       )

if __name__ == "__main__":
    demo.queue().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1786774fb47e4169f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def yes_man(message, history):
    if message.endswith("?"):
        return "Yes"
    else:
        return "Ask me anything!"

gr.ChatInterface(
    yes_man,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7),
    title="Yes Man",
    description="Ask Yes Man any question",
    theme="soft",
    examples=["Hello", "Am I cool?", "Are tomatoes vegetables?"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

Caching examples at: '/content/gradio_cached_examples/201'
Caching example 1/3
Caching example 2/3
Caching example 3/3
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://359c81fee63337403a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##### Liking / Disliking Chat Messages

In [ ]:
#from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

#llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    gpt_response = conversational_qa_chain.invoke(
        {
          "question": message,
          "chat_history": history_langchain_format
        })
    return gpt_response.content


def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value)
    else:
        print("You downvoted this response: " + data.value)


#gr.ChatInterface(predict,
#                 title='MCO Assistant',
#                 description='A generative AI assistant which can enable use cases such as customer or field support, employee training and developer productivity.\n You can use the examples provided to get started. This model has the ability to answer the questions in its orignal language.',
#                 textbox=gr.Textbox(placeholder="Type your question here..."),
#                 examples=["What does MCO provide?", "C'est facile?", "Are tomatoes vegetables?"],
#                 theme='soft'
#                 ).launch()


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    textbox = gr.Textbox()
    textbox.submit(greet, [chatbot, textbox], [chatbot])
    chatbot.like(vote, None, None)  # Adding this line causes the like/dislike icons to appear in your chatbot

demo.launch()

DuplicateBlockError: A block with id: 23 has already been rendered in the current Blocks.

In [ ]:
import gradio as gr

def greet(history, input):
    return history + [(input, "Hello, " + input)]

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value)
    else:
        print("You downvoted this response: " + data.value)


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    textbox = gr.Textbox()
    textbox.submit(greet, [chatbot, textbox], [chatbot])
    chatbot.like(vote, None, None)  # Adding this line causes the like/dislike icons to appear in your chatbot

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://25e0e30865a0577475.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
